# Import

In [1]:
import csv
import os
import pickle
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")
sys.path.append("../PlotFunction/lineplot/")
sys.path.append("../PlotFunction/config/")

In [3]:
from ALL import config
from line_plot_1 import line_plot_1
from line_plot_error_1 import line_plot_error_1
from line_plot_1_layout import layout
from util import *

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [5]:
data_types = ["AgNews"]
vectorize_types = ["sentenceBERT"]

In [6]:
# model_nums = config["clustering"]["gmm"]["max_model_num"]
model_nums = 10
# covariance_types = config["clustering"]["gmm"]["covariance_types"]
normalization = "normalized"
covariance_types = ["spherical"]
vector_dims = {
    "sentenceBERT": [20]
}
topic_nums = [2, 4, 8, 16, 32, 64, 128]

In [7]:
stats_vals = ["aic", "bic", "mi", "logl"]

# Stats

## Read data

In [13]:
stats = multilayer_dict(
    {
        "stats_val": stats_vals,
        "covariance_types": covariance_types,
        "model_nums": range(model_nums),
        "topic_num": topic_nums,
    }
)

In [8]:
def load_stats_data(
    vectorize_type, stats_vals, covariance_types, model_nums, topic_nums
):
    # データ型定義
    stats = make_multilayer_dict(
        [stats_vals, covariance_types, range(model_nums), topic_nums]
    )

    # データ取得
    for stats_val in stats_vals:
        for covariance_type in covariance_types:
            for model_num in range(model_nums):
                for topic_num in topic_nums:
                    stats_path = f"../../Postprocessing/data/{data_type}/{vectorize_type}/GMM/stats/{normalization}/{covariance_type}/{topic_num}/{model_num}.csv"
                    df = pd.read_csv(stats_path, index_col=0)
                    stats[stats_val][covariance_type][model_num] = df.loc[:, stats_val]
    return stats

In [8]:
def load_stats_data(vectorize_type, stats_vals, covariance_types, model_nums, topic_nums):
    # データ型定義
    stats = {
        stats_val: {
            covariance_type: {
                model_num: {topic_num: pd.DataFrame for topic_num in topic_nums}
                for model_num in range(model_nums)
            }
            for covariance_type in covariance_types
        }
        for stats_val in stats_vals
    }

    # データ取得
    for stats_val in stats_vals:
        for covariance_type in covariance_types:
            for model_num in range(model_nums):
                for topic_num in topic_nums:
                    stats_path = f"../../Postprocessing/data/{data_type}/{vectorize_type}/GMM/stats/{normalization}/{covariance_type}/{topic_num}/{model_num}.csv"
                    df = pd.read_csv(stats_path, index_col=0)
                    stats[stats_val][covariance_type][model_num] = df.loc[:, stats_val]
    return stats

In [9]:
def load_lda_mi(data_type):
    lda_mi = pd.read_csv(
        f"../../Postprocessing/data/{data_type}/LDA/mi.csv", index_col=0
    )
    describe_lda_mi, _ = get_describe(lda_mi, axis=0)
    return describe_lda_mi

In [10]:
stats_dict = {}
for vectorize_type in vectorize_types:
    for data_type in data_types:
        stats_dict[vectorize_type] = load_stats_data(
            vectorize_type, stats_vals, covariance_types, model_nums, topic_nums
        )

NameError: name 'make_multilayer_dict' is not defined

In [11]:
describe_lda_mi = load_lda_mi(data_type)

## Data shaping

In [24]:
def shape_stats_df(stats):
    stats_df = {
        stats_val: {covariance_type: pd.DataFrame() for covariance_type in covariance_types}
        for stats_val in stats_vals
    }

    for stats_val in stats_vals:
        for covariance_type in covariance_types:
            # model_numについて取得データstatsを結合
            stats_df[stats_val][covariance_type] = pd.concat(
                stats[stats_val][covariance_type], axis=1
            )
    return stats_df

In [25]:
def shape_describe(stats_df):
    describe = {
        stats_val: {covariance_type: dict() for covariance_type in covariance_types}
        for stats_val in stats_vals
    }

    for stats_val in stats_vals:
        for covariance_type in covariance_types:
            describe[stats_val][covariance_type], describe_keys = get_describe(
                stats_df[stats_val][covariance_type], axis=1
            )
    return describe, describe_keys

In [26]:
def shape_data(describe, describe_keys):
    data = {
        stats_val: {describe_key: pd.DataFrame() for describe_key in describe_keys}
        for stats_val in stats_vals
    }
    for stats_val in stats_vals:
        for describe_key in describe_keys:
            # covariance_typeについてデータを結合
            _data = {
                covariance_type: describe[stats_val][covariance_type][describe_key]
                for covariance_type in covariance_types
            }
            data[stats_val][describe_key] = pd.concat(_data, axis=1)
    return data

In [73]:
describes = {
    vectorize_type: {
        stats_val: {covariance_type: {} for covariance_type in covariance_types}
        for stats_val in stats_vals
    }
    for vectorize_type in vectorize_types
}

for vectorize_type in vectorize_types:
    for stats_val in stats_vals:
        for covariance_type in covariance_types:
            df[covariance_type] = pd.concat(
                stats_dict["sentenceBERT"][stats_val][covariance_type], axis=1
            )
            (
                describes[vectorize_type][stats_val][covariance_type],
                describe_key,
            ) = get_describe(df[covariance_type], axis=1)

In [459]:
data = {2: ["a", "b", "c"], 1: ["d", "c"], 3: ["e", "f"], 4: ["g"]}

In [462]:
multi_d = multilayer_dict(data)

# Make Chart

In [32]:
chart_data = {}
chart_data["sentenceBERT"] = data_stats["sentenceBERT"]["mi"]["mean"].loc[384, :]

KeyError: 384

In [33]:
pd.DataFrame(chart_data)

""


In [132]:
chart_df = pd.DataFrame(chart_data)

In [133]:
chart_df.T.style.format(
    escape="latex", formatter={"document_count": "{:.0f}"}
)

,spherical,diag,full
doc2vec,0.454906,0.429935,0.475413
sentenceBERT,0.582227,0.585635,0.607587


In [134]:
chart_df.to_csv(make_filepath(f"../data/{data_type}/CovarianceChart.csv"))

In [135]:
print(
    chart_df.T
    .style.format(precision=3, escape="latex")
    .to_latex(
        column_format="rrrr",
        position="h",
        position_float="centering",
        hrules=True,
        caption="miの比較と埋め込み次元",
        label="table:1",
        multicol_align="r",
    )
)

\begin{table}[h]
\centering
\caption{miの比較と埋め込み次元}
\label{table:1}
\begin{tabular}{rrrr}
\toprule
 & spherical & diag & full \\
\midrule
doc2vec & 0.455 & 0.430 & 0.475 \\
sentenceBERT & 0.582 & 0.586 & 0.608 \\
\bottomrule
\end{tabular}
\end{table}

